### MPL Classifier


We are creating an MLP classifier with 3 hidden layers.

Use the below code to install torch vision (uncomment the below line and execute it).

In [1]:
#!pip3 install torch==1.10.1+cu102 torchvision==0.11.2+cu102 torchaudio===0.10.1+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

In [26]:
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")
ipython = get_ipython()

In [27]:
# import libraries
from matplotlib import pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

print("All libraries are loaded")

All libraries are loaded


In [28]:
import torch
import random
import numpy as np
import pandas as pd

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(True)


In [34]:
# how many samples per batch to load
batch_size = 15

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

#Loading train data
train_data_load = pd.read_csv("iris_training.csv", header=None)

#transforming train data
train_input_columns = torch.from_numpy(train_data_load[list(train_data_load.columns)[:-1]].to_numpy()).type(torch.float32)
train_output_columns = torch.tensor(train_data_load[list(train_data_load.columns)[-1]])

#Loading test data
test_data_load = pd.read_csv("iris_test.csv", header=None)

#transforming test data
test_input_columns = torch.from_numpy(test_data_load[list(test_data_load.columns)[:-1]].to_numpy()).type(torch.float32)
test_output_columns = torch.tensor(test_data_load[list(test_data_load.columns)[-1]])

# choose the training and test datasets
train_data = torch.utils.data.TensorDataset(train_input_columns,train_output_columns)
#train_data = pd.read_csv("iris_training.csv", header=None)

print(len(train_data))

train_data, val_data = torch.utils.data.random_split(train_data, [100, 20])

test_data = torch.utils.data.TensorDataset(test_input_columns,test_output_columns)
#test_data = pd.read_csv("iris_test.csv", header=None)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
print("Training data created")
print("Train-Data length : ", len(train_data),"\n")


val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
print("Validation data created")
print("Validation-Data length : ", len(val_data),"\n")


test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
print("Test data created")
print("Test-Data length : ", len(test_data))



120
Training data created
Train-Data length :  100 

Validation data created
Validation-Data length :  20 

Test data created
Test-Data length :  30


In [35]:
## Define the NN architecture

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        """
        Defining Hidden Layer 1 (Linear Layer)
        As the no of input parameters are 4 hence we are providing input as 4. 
        For current layer we are setting the system to provide an output of 10 features"""
        self.fc1 = nn.Linear(4, 10)
        
        """
        Defining Hidden Layer 2 (Linear Layer)
        As the number of output parameter of previous hidden layer was 10, hence the input of current hidden layer will be same.
        Similar to previous layer we are not setting output features to 5
        """
        self.fc2 = nn.Linear(10, 5)       
        
        """
        Defining Hidden Layer 3 (Linear Layer)
        """
        self.fc3 = nn.Linear(5, 3)        

    def forward(self, x):
        # Adding hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

# initialize the NN
model = Net()

# Printing the model
print(model)

Net(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=3, bias=True)
)


In [36]:
# Specify loss and optimization functions

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

print(criterion, optimizer)

CrossEntropyLoss() SGD (
Parameter Group 0
    dampening: 0
    lr: 0.05
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [37]:
# Declaring varible to define number of epochs to train the model.
# We are setting the epochs to 10
n_epochs = 20 

# Preparing model for training
model.train() 

# Executing epochs one by one 
for epoch in range(n_epochs):
    
    # Declaring and Initializing variables for training and validation loss with 0 value
    train_loss = 0.0                    
    val_loss=0.0
    
    # train the model 
    for data, target in train_loader:
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()            
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)             
        
        # calculate the loss
        loss = criterion(output, target) 
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()                  
        
        # perform a single optimization step to update the parameters
        optimizer.step()                 
        
        # update running training loss
        train_loss += loss.item()*data.size(0)  
    
    
    
    for data, target in val_loader:
        
         # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)                 
        
         # calculate the loss
        loss = criterion(output, target)     
        
        # update running validation loss
        val_loss += loss.item()*data.size(0)  
        
    
    # calculate average loss over an epoch"""
    train_loss = train_loss/len(train_loader.dataset)  
    
    # Validation loss 
    val_loss = val_loss/len(val_loader.dataset)        

    # print training statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format( epoch+1, train_loss, val_loss ))
    print("Training is done\n")

Epoch: 1 	Training Loss: 1.084156 	Validation Loss: 1.074800
Training is done

Epoch: 2 	Training Loss: 1.077249 	Validation Loss: 1.070323
Training is done

Epoch: 3 	Training Loss: 1.075868 	Validation Loss: 1.066130
Training is done

Epoch: 4 	Training Loss: 1.068739 	Validation Loss: 1.063694
Training is done

Epoch: 5 	Training Loss: 1.068762 	Validation Loss: 1.062281
Training is done

Epoch: 6 	Training Loss: 1.064346 	Validation Loss: 1.058054
Training is done

Epoch: 7 	Training Loss: 1.059958 	Validation Loss: 1.058377
Training is done

Epoch: 8 	Training Loss: 1.058690 	Validation Loss: 1.053769
Training is done

Epoch: 9 	Training Loss: 1.054953 	Validation Loss: 1.054883
Training is done

Epoch: 10 	Training Loss: 1.051850 	Validation Loss: 1.049729
Training is done

Epoch: 11 	Training Loss: 1.049080 	Validation Loss: 1.045427
Training is done

Epoch: 12 	Training Loss: 1.045580 	Validation Loss: 1.041114
Training is done

Epoch: 13 	Training Loss: 1.040842 	Validation Lo

In [38]:
# Testing the  model 
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for *evaluation*

for data, target in test_loader:
    output = model(data)                            # forward pass: compute predicted outputs by passing inputs to the model
    loss = criterion(output, target)                # calculate the loss
    test_loss += loss.item()*data.size(0)           # update test loss 
    _, pred = torch.max(output, 1)                  # convert output probabilities to predicted class
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))        # compare predictions to true label
    
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(3):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
print()

Test Loss: 1.072049

Test Accuracy of     0: 100% ( 8/ 8)
Test Accuracy of     1:  0% ( 0/14)
Test Accuracy of     2: 100% ( 8/ 8)

Test Accuracy (Overall): 53% (16/30)

